In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('\tmp\data', one_hot=True)

Extracting 	mp\data/train-images-idx3-ubyte.gz
Extracting 	mp\data/train-labels-idx1-ubyte.gz
Extracting 	mp\data/t10k-images-idx3-ubyte.gz
Extracting 	mp\data/t10k-labels-idx1-ubyte.gz


In [13]:
learning_rate = 0.01
batch_size = 128
display_step = 1
model_path = "/tmp/model.ckpt"

num_hidden1 = 256
num_hidden2 = 256
num_input = 28*28
num_classes = 10

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [14]:
weights = {
    'w1': tf.Variable(tf.random_normal([num_input, num_hidden1])),
    'w2': tf.Variable(tf.random_normal([num_hidden1, num_hidden2])),
    'out': tf.Variable(tf.random_normal([num_hidden2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden1])),
    'b2': tf.Variable(tf.random_normal([num_hidden2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def multilayer_net(x, weights, biases):
    layer1 = tf.matmul(x, weights['w1']) + biases['b1']
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.matmul(layer1, weights['w2']) + biases['b2']
    layer2 = tf.nn.relu(layer2)
    
    out = tf.matmul(layer2, weights['out']) + biases['out']
    return out

pred = multilayer_net(X, weights, biases)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [15]:
saver = tf.train.Saver()

In [20]:
print("1st session")
with tf.Session() as sess:
    
    sess.run(init)
    
    for epoch in range(3):
        ave_cost = 0
        num_steps = int(mnist.train.num_examples/batch_size)
        for step in range(num_steps):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, loss = sess.run([train_op, loss_op], feed_dict = {X:batch_x, Y:batch_y})
        
            ave_cost += loss/num_steps
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(ave_cost))
            
    accu = sess.run([accuracy], feed_dict = {X: mnist.test.images, Y: mnist.test.labels})
    print("accuracy", accu)
    
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)

1st session
Epoch: 0001 cost= 57.437561506
Epoch: 0002 cost= 10.846982995
Epoch: 0003 cost= 5.591072588
accuracy [0.9457]
Model saved in file: /tmp/model.ckpt


In [ ]:
print("2nd session")
with tf.Session() as sess:
    
    sess.run(init)
    
    load_path = saver.restore(sess, model_path)
    for epoch in range(7):
        ave_cost = 0
        num_steps = int(mnist.train.num_examples/batch_size)
        for step in range(num_steps):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, loss = sess.run([train_op, loss_op], feed_dict = {X:batch_x, Y:batch_y})
        
            ave_cost += loss/num_steps
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(ave_cost))
            
    accu = sess.run([accuracy], feed_dict = {X: mnist.test.images, Y: mnist.test.labels})
    print("accuracy", accu)
    
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)